#### 1. 卷积样例

In [3]:
import tensorflow as tf

In [4]:
# 1) 图片处理

# input输入图像是一个四维矩阵
# 第1维是第X个图像
# 第2第3维是长度宽度为32*32
# 第4维是RGB颜色通道数为3的矩阵
input = tf.placeholder(
    name='input',
    shape=[None, 32, 32, 3], 
    dtype=tf.float32
)

# 2) 卷积核的weight
# filter是一个四维矩阵
# 第1第2维是过滤器尺寸width和height（5*5），
# 第3维当前层深度（3）
# 第4维是过滤器深度（16）
# 初始化为正态分布的随机变量
filter_weight = tf.get_variable(
    name='weights', 
    shape=[5, 5, 3, 16], 
    initializer=tf.truncated_normal_initializer(stddev=0.1)
)

# biases偏置量是一个一维矩阵
# 第1维是偏置量的深度（16）
# 初始化为数值都是0.1的一维矩阵
biases = tf.get_variable(
    name='biases', 
    shape=[16], 
    initializer=tf.constant_initializer(0.1)
)

# 3) 建立卷积层  第一个卷积层 c onv=conv2d(input,filter_weight)+biases

# tf.nn.conv2d的strides步长参数
# strides步长是一个四维矩阵
# 第1维和第4维必须为1，因为卷积层的步长只对矩阵的长宽有效
# 第2维和第3维是长和宽的步长
# 
# tf.nn.conv2d的padding参数只提供两种SAME以及VAILD
# SAME表示添加0填充
# VAILD表示不添加填充
conv = tf.nn.conv2d(
    input=input, 
    filter=filter_weight, 
    strides=[1,1,1,1], 
    padding='SAME'
)

# tf.nn.bias_add提供了一个方便的函数给节点添加偏置量（注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置量）
# 例如下一层神经网络的大小为2*2，但是偏置量只有一个数（因为深度为1），而2*2矩阵的每一个项都需要加上这个偏置量。
bias = tf.nn.bias_add(value=conv, bias=biases)

# 4） 激活函数，这里我们用的是tf.nn.relu（修正线性单元）来处理
# 把计算结果通过relu激活函数进行去线性化
actived_conv = tf.nn.relu(features=bias)


要注意的是，因为采用了SAME的padding方式，输出图片的大小没有变化依然是32x32，只是厚度变厚了

#### 2. 池化样例

In [5]:
# 池化有两种常用的池化
# tf.nn.max_pool最大值池化
# tf.nn.avg_pool均值池化

# ksize为池化层过滤器filter的尺寸，过滤器的头尾两个维度必须等于1
# 实际中使用最多的尺寸是[1,2,2,1]或者[1,3,3,1]
# strides为池化层过滤器步长
pool = tf.nn.max_pool(
    value=actived_conv, 
    ksize=[1,3,3,1], 
    strides=[1,2,2,1], 
    padding='SAME'
)


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(biases))
    print(sess.run(filter_weight))

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[[[[ 5.92819750e-02  2.02158373e-02 -1.65881608e-02 ...  1.47966072e-01
     6.88818619e-02 -6.43078098e-03]
   [-1.10610165e-01  1.44120947e-01 -1.04992390e-01 ... -4.27408032e-02
    -9.54718515e-02 -8.14912096e-02]
   [-6.41070157e-02 -2.12700497e-02  1.40515929e-02 ...  4.52333651e-02
     1.03443801e-01 -1.11757211e-01]]

  [[ 9.33207348e-02 -4.25871126e-02 -6.13673292e-02 ...  1.60967503e-02
     2.37542223e-02 -4.78126341e-03]
   [ 7.32640773e-02 -1.74460426e-01 -4.44172435e-02 ... -1.12216569e-01
    -4.36930880e-02  1.13553979e-01]
   [ 7.25990832e-02  1.37484178e-01  2.57125832e-02 ... -4.59335558e-02
     1.91340726e-02 -5.52500561e-02]]

  [[-1.51204363e-01 -8.68785355e-05  8.56378898e-02 ... -5.52387312e-02
     4.53325771e-02 -1.74263313e-01]
   [-1.81325778e-01  1.28603727e-02 -1.33465990e-01 ...  1.52940825e-01
     8.79561603e-02  1.17710903e-01]
   [-1.18048927e-02  4.86791059e-02  1.72945380e-01 ...  2

#### LENET-5模型

```python
input: [width=32, height=32, color=1]

conv1:
filter: [width=5, height=5, input-dims=1, output-dims=6]
stride: [x-stride=1, y-stride=1]
output: [width=(32-5+1)/1=28, height=(32-5+1)/1=28, output-dims=6]

pool1:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=28/2=14, height=28/2=14, output-dims=6]

conv2:
filter: [width=5, height=5, input-dims=6, output-dims=16]
stride: [x-stride=1, y-stride=1]
output: [width=(14-5+1)/1=10, height=(14-5+1)/1=10, output-dims=16]

pool2:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=10/2=5, height=10/2=5, output-dims=16]

full-connect1:
w: [5*5*16=400, 120]
output: [120]

full-connect2:
w: [120, 84]
output: [84]

full-connect3:
w: [84, 10]
output: [10]
```